다음의 커널을 필사했습니다.  
https://www.kaggle.com/jsaguiar/lightgbm-7th-place-solution/data  
https://www.youtube.com/watch?v=aoo1xrKQXFc&list=PLC_wC_PMBL5PexxXb6UC6pDXODE4xFWzL

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
import gc
import time
import numpy as np
import pandas as pd
from contextlib import contextmanager
import multiprocessing as mp
from functools import partial
from scipy.stats import kurtosis, iqr, skew
from lightgbm import LGBMClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
import warnings

In [3]:
warnings.simplefilter(action='ignore', category = FutureWarning)

if __name__ == "__main__":  
    pd.set_option('display.max_rows', 60)  
    pd.set_option('display.max_columns', 100)  
    with timer("Pipeline total time"):  
        main(debug= False)  
        
위의 코드로 실행이 되는데 이 노트북에서는 스크립트 형으로 진행하지 않기 때문에 처음부터 설정을 해줍니다.

In [4]:
pd.set_option('display.max_rows', 60)

In [5]:
pd.set_option('display.max_columns', 100)

In [6]:
# main 함수에서 debug = False로 돼있지만 지금은 True이기 때문에 인위적으로 True로 해줍니다.

debug = True
num_rows = 30000 if debug else None

## df = get_train_test(DATA_DIRECTORY, num_rows = num_rows) # 함수 뜯어보기

In [7]:
# GENERAL CONFIGURATIONS
NUM_THRADS = 4
DATA_DIRECTORY = "../input/"
SUBMISSION_SUFIX = "_model2_04"

In [8]:
path = DATA_DIRECTORY
num_rows = num_rows

In [9]:
train_file_path = '../input/home-credit-default-risk/application_train.csv'
test_file_path = '../input/home-credit-default-risk/application_test.csv'

In [10]:
train = pd.read_csv(train_file_path, nrows = num_rows)
test = pd.read_csv(test_file_path, nrows = num_rows)

In [11]:
df = train.append(test)

In [12]:
print(train.shape)

In [13]:
train.head()

In [14]:
train['TARGET'].value_counts().plot.bar()

In [15]:
del train, test

# 메모리 주워 담기
gc.collect

가장 먼저 target값이 imbalance한지 봐야합니다.

# Preprocessing

원래는 EDA를 먼저 들어가야 하지만 여기서는 해당 파트에 대한 설명이 없기 때문에 Preprocessing에 바로 들어갑니다.

In [16]:
# 전체 데이터 프레임을 봤을 때 XNA에는 4명의 사람 밖에 없기 때문에 제거합니다. (아웃라이어로 판단)
df = df[df['CODE_GENDER'] != 'XNA']

In [17]:
# 2천만 이하는 100개 정도 밖에 없기 때문에 이 또한 제거합니다.
df = df[df['AMT_INCOME_TOTAL'] < 20000000]

365243으로 기록된 사람들이 많은데, 이는 의미 없는 숫자로 보입니다. 하지만 이를 제거하면 문제가 생길 수 있습니다. 따라서 일단 nan값을 적용하겠습니다.

In [18]:
df

In [19]:
df[df['DAYS_EMPLOYED'] == 365243].value_counts()

In [20]:
df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)

In [21]:
# 핸드폰 바꾼지 0일이라는 것은 null 값을 무언가로 채웠음을 의미하는 것 같습니다. 따라서 nan으로 대체합니다.
df['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True)

In [22]:
# 이들은 binary data입니다. "에, 아니오" 등으로 이루어진 문서
docs = [f for f in df.columns if 'FLAG_DOC' in f]

In [23]:
docs

In [24]:
df[docs].kurtosis(axis=1).hist()

대부분이 특정한 값으로 몰려 있습니다.

In [25]:
# 설문조사에서 "예"를 몇 번 했는지 체크
df['DOCUMENT_COUNT'] = df[docs].sum(axis=1)

# 왜곡된 정도를 컬럼으로 추가
df['NEW_DOC_KURT'] = df[docs].kurtosis(axis=1) 

get_age_label 함수의 df['DAYS_BIRTH']를 보면 -살아온 일 수가 기록돼 있습니다.  
여기다 -를 취해 양수로 만들어준 이후 365로 나누면 나이가 나오는데 이를 구간으로 나눠주는 함수를 만듭니다.

In [26]:
df['DAYS_BIRTH'][:5]

continuos한 정보를 categorical한 것으로 바꿔줍니다.

In [27]:
def get_age_label(days_birth):
    """ Return the age group label (int). """
    age_years = -days_birth / 365
    if age_years < 27: return 1
    elif age_years < 40: return 2
    elif age_years < 50: return 3
    elif age_years < 65: return 4
    elif age_years < 99: return 5
    else: return 0

In [28]:
df['AGE_RANGE'] = df['DAYS_BIRTH'].apply(lambda x: get_age_label(x))

EXT_SOURCE는 feature importance가 높게 나오는 값들입니다. 이들끼리 서로 interaction할 수 있게 곱해서 새로운 feature를 만들어줍니다.  
현업에서는 설명이 되지 않는 변수는 사용하기 어렵지만 캐글에서는 점수를 높여야 하니 사용하는 경향이 있습니다.

In [29]:
df['EXT_SOURCES_PROD'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']

경진대회에 참여하는 사람들이 해당 feature들 간의 상관관계를 조사해본 결과 2, 1, 3이 좋게 나타났습니다. 따라서 이러한 피처를 추가해줍니다.

In [30]:
df['EXT_SOURCES_WEIGHTED'] = df.EXT_SOURCE_1 * 2 + df.EXT_SOURCE_2 * 1 + df.EXT_SOURCE_3 * 3

eval() 함수는 np.min, np.var과 같은 통계값을 string 형태로 끼워 넣을 수 있도록 만들어줍니다. 이렇게 하면 다양한 통계값을 직접 손으로 쳐야 하는 상황을 단순하게 만들 수 있다는 장점이 있습니다.

In [31]:
for function_name in ['min', 'max', 'mean', 'nanmedian', 'var']:
    feature_name = 'EXT_SOURCES_{}'.format(function_name.upper())
    # 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3' 등의 통계값을 구하는 코드
    df[feature_name] = eval('np.{}'.format(function_name))(
        df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']], axis = 1)

지금 만드는 피처들은 어느 정도의 도메인 지식이 필요합니다.

In [32]:
# 연금을 받는데 빛은 얼마나 지는지
df['CREDIT_TO_ANNUITY_RATIO'] = df['AMT_CREDIT'] / df['AMT_ANNUITY']
# 물건을 사는데 빚을 얼마나 지는지
df['CREDIT_TO_GOODS_RATIO'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']

In [33]:
# Income ratios
df['ANNUITY_TO_INCOME_RATIO'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
df['CREDIT_TO_INCOME_RATIO'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']
 # 일한 햇수에 비해 임금이 얼만지
df['INCOME_TO_EMPLOYED_RATIO'] = df['AMT_INCOME_TOTAL'] / df['DAYS_EMPLOYED']
df['INCOME_TO_BIRTH_RATIO'] = df['AMT_INCOME_TOTAL'] / df['DAYS_BIRTH']

In [34]:
# Time ratios
# 살아온 날 대비 얼마나 근로했는지
df['EMPLOYED_TO_BIRTH_RATIO'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
# 회사에 아이디 만들어진지 얼마나 되는지
df['ID_TO_BIRTH_RATIO'] = df['DAYS_ID_PUBLISH'] / df['DAYS_BIRTH']
# 나이 대비 car를 소유한 일 수
df['CAR_TO_BIRTH_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_BIRTH']
df['CAR_TO_EMPLOYED_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_EMPLOYED']
df['PHONE_TO_BIRTH_RATIO'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH']

# do_mean 등 통계값 구하는 함수 뜯어보기

In [35]:
group = ['ORGANIZATION_TYPE', 'NAME_EDUCATION_TYPE', 'OCCUPATION_TYPE', 'AGE_RANGE', 'CODE_GENDER']

In [36]:
df, group_cols, counted, agg_name = df, group, 'EXT_SOURCES_MEAN', 'GROUP_EXT_SOURCES_MEAN'

In [37]:
# counted가 밑으로 추가됨
group_cols + [counted]

In [38]:
# 목표로 하는 컬럼만 가져옴
df[group_cols + [counted]].head()

df에서 원하는 컬럼만을 가져오고, groupby 이후 통계치를 얻어냅니다. 이후 `reset_index`를 통해 모양을 바로 잡고, 컬럼 이름이 겹치지 않게 rename을 적용해 줍니다.

In [39]:
df[group_cols + [counted]].groupby(group_cols)[counted].mean().reset_index().rename(columns={counted: agg_name}).head()

위의 작업을 수행한 데이터 프레임을 gp 이름으로 넣어줍니다. 이후 원래 df와 gp를 merge하는데, 두 데이터 프레임이 공통적으로 가지고 있는 group_cols를 기준으로 합쳐줍니다.

In [40]:
def do_mean(df, group_cols, counted, agg_name):
    gp = df[group_cols + [counted]].groupby(group_cols)[counted].mean().reset_index().rename(
        columns = {counted: agg_name})
    df = df.merge(gp, on = group_cols, how = 'left')
    del gp
    gc.collect()
    return df

def do_median(df, group_cols, counted, agg_name):
    gp = df[group_cols + [counted]].groupby(group_cols)[counted].median().reset_index().rename(
        columns = {counted: agg_name})
    df = df.merge(gp, on = group_cols, how = 'left')
    del gp
    gc.collect()
    return df

def do_std(df, group_cols, counted, agg_name):
    gp = df[group_cols + [counted]].groupby(group_cols)[counted].std().reset_index().rename(
        columns = {counted: agg_name})
    df = df.merge(gp, on = group_cols, how = 'left')
    del gp
    gc.collect()
    return df

def do_sum(df, group_cols, counted, agg_name):
    gp = df[group_cols + [counted]].groupby(group_cols)[counted].sum().reset_index().rename(
        columns = {counted: agg_name})
    df = df.merge(gp, on = group_cols, how = 'left')
    del gp
    gc.collect()
    return df

In [41]:
df = do_median(df, group, 'EXT_SOURCES_MEAN', 'GROUP_EXT_SOURCES_MEDIAN')
df = do_std(df, group, 'EXT_SOURCES_MEAN', 'GROUP_EXT_SOURCES_STD')
df = do_mean(df, group, 'AMT_INCOME_TOTAL', 'GROUP_INCOME_MEAN')
df = do_std(df, group, 'AMT_INCOME_TOTAL', 'GROUP_INCOME_STD')
df = do_mean(df, group, 'CREDIT_TO_ANNUITY_RATIO', 'GROUP_CREDIT_TO_ANNUITY_MEAN')
df = do_std(df, group, 'CREDIT_TO_ANNUITY_RATIO', 'GROUP_CREDIT_TO_ANNUITY_STD')
df = do_mean(df, group, 'AMT_CREDIT', 'CROUP_CREDIT_MEAN')
df = do_mean(df, group, 'AMT_ANNUITY', 'GROUP_ANNUITY_MEAN')
df = do_std(df, group, 'AMT_ANNUITY', 'GROUP_ANNUITY_STD')

## Encode categorical features (LabelEncoder) | 라벨인코더 뜯어보기
df, le_encoded_cols = label_encoder(df, None)  
df = drop_application_columns(df)  
return df

판다스는 문자열을 object라는 자료형으로 나타냅니다. 데이터프레임을 만들 때 dtype을 지정해 주는 것이 아니라면 숫자형을 제외한 나머지는 object로 받아들입니다. 이들을 인코딩 해줍니다.  
참고:  
https://abluesnake.tistory.com/101

In [42]:
for i in df.columns:
    if df[i].dtype == object:
        print(i)

In [43]:
df = df
categorical_columns = None

if not categorical_columns:
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']

In [44]:
categorical_columns

In [45]:
for col in categorical_columns:
    break

factorize 함수는 0, 1로 인코딩 된 결과, 그리고 이에 대한 인덱스를 반환합니다.

In [46]:
pd.factorize(df[col])

In [47]:
def label_encoder(df, categorical_columns=None):
    """Encode categorical values as integers (0,1,2,3...) with pandas.factorize. """
    if not categorical_columns:
        categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    for col in categorical_columns:
        df[col], uniques = pd.factorize(df[col])
    return df, categorical_columns

In [48]:
df, le_encoded_cols = label_encoder(df, None)

null importances를 바탕으로 피처를 선택하며 중요도가 낮은 피처는 제거합니다.  
참고:  
https://www.kaggle.com/ogrellier/feature-selection-with-null-importances  
https://databreak.netlify.app/2019-04-21-Null_importance/

In [49]:
for i in df.columns:
    if 'FLAG_DOCUMENT' in i:
        print(i)

In [50]:
def drop_application_columns(df):
    """ Drop features based on permutation feature importance. """
    drop_list = [
        'CNT_CHILDREN', 'CNT_FAM_MEMBERS', 'HOUR_APPR_PROCESS_START',
        'FLAG_EMP_PHONE', 'FLAG_MOBIL', 'FLAG_CONT_MOBILE', 'FLAG_EMAIL', 'FLAG_PHONE',
        'FLAG_OWN_REALTY', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION',
        'REG_CITY_NOT_WORK_CITY', 'OBS_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE',
        'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_YEAR', 
        'COMMONAREA_MODE', 'NONLIVINGAREA_MODE', 'ELEVATORS_MODE', 'NONLIVINGAREA_AVG',
        'FLOORSMIN_MEDI', 'LANDAREA_MODE', 'NONLIVINGAREA_MEDI', 'LIVINGAPARTMENTS_MODE',
        'FLOORSMIN_AVG', 'LANDAREA_AVG', 'FLOORSMIN_MODE', 'LANDAREA_MEDI',
        'COMMONAREA_MEDI', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'BASEMENTAREA_AVG',
        'BASEMENTAREA_MODE', 'NONLIVINGAPARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 
        'LIVINGAPARTMENTS_AVG', 'ELEVATORS_AVG', 'YEARS_BUILD_MEDI', 'ENTRANCES_MODE',
        'NONLIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 'LIVINGAPARTMENTS_MEDI',
        'YEARS_BUILD_MODE', 'YEARS_BEGINEXPLUATATION_AVG', 'ELEVATORS_MEDI', 'LIVINGAREA_MEDI',
        'YEARS_BEGINEXPLUATATION_MODE', 'NONLIVINGAPARTMENTS_AVG', 'HOUSETYPE_MODE',
        'FONDKAPREMONT_MODE', 'EMERGENCYSTATE_MODE'
    ]
        
    # Drop most flag document columns
    for doc_num in [2, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]:
        drop_list.append('FLAG_DOCUMENT_{}'.format(doc_num))
    df.drop(drop_list, axis=1, inplace=True)
    return df

In [51]:
df = drop_application_columns(df)

In [52]:
df

# get_bureau 함수 뜯어보기

bureau는 당사의 데이터가 아닌 외부 신용평가 기관으로부터 고객의 정보를 가져온 것을 의미합니다.

In [53]:
bureau_path = '../input/home-credit-default-risk/bureau.csv'
bureau = pd.read_csv(bureau_path, nrows=num_rows)

In [54]:
# Credit duration and credit/account end date difference
bureau['CREDIT_DURATION'] = bureau['DAYS_CREDIT'] + bureau['DAYS_CREDIT_ENDDATE']
bureau['ENDDATE_DIF'] = bureau['DAYS_CREDIT_ENDDATE'] - bureau['DAYS_ENDDATE_FACT']

# Credit to debt ratio and difference
bureau['DEBT_PERCENTAGE'] = bureau['AMT_CREDIT_SUM'] / bureau['AMT_CREDIT_SUM_DEBT']
bureau['DEBT_CREDIT_DIFF'] = bureau['AMT_CREDIT_SUM'] - bureau['AMT_CREDIT_SUM_DEBT']
bureau['CREDIT_TO_ANNUITY_RATIO'] = bureau['AMT_CREDIT_SUM'] / bureau['AMT_ANNUITY']

In [55]:
# One hot encoder
def one_hot_encoder(df, categorical_columns=None, nan_as_category=True):
    original_columns = list(df.columns)
    if not categorical_columns:
        categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    # categorical_columns 만드는 과정 생략. 위에서 만든 자료 사용했음
    # dummy_na > nan 값을 dummies를 만들 때 포함할 것인지의 여부
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)
    categorical_columns = [c for c in df.columns if c not in original_columns]
    return df, categorical_columns

null data를 nan으로 채우지 않았는데, 그 이유는 LightGBM에서 자동적으로 이러한 값들을 채워주기 때문입니다.

In [56]:
bureau, categorical_cols = one_hot_encoder(bureau, nan_as_category = False)

In [57]:
bureau_balance_path = '../input/home-credit-default-risk/bureau_balance.csv'
bb = pd.read_csv(bureau_balance_path, nrows=num_rows)

In [58]:
bb.head()

breau_balance_STATUS는 loan의 상태를 말해줍니다. (active인지 closed인지 등..)

In [59]:
bb, categorical_cols = one_hot_encoder(bb, nan_as_category = False)

# get_bureau_balance 함수 뜯어보기

`get_bureau_balance` 함수 안에는 `group_and_merge`함수가 있으며 이 안에는 또 `group`함수가 있습니다. `get_bureau_balance`를 이해하기 위해서는 이들이 어떻게 동작하는지 알아야 합니다.

In [60]:
def get_bureau_balance(path, num_rows= None):
    bb = pd.read_csv(bureau_balance_path , nrows= num_rows)
    bb, categorical_cols = one_hot_encoder(bb, nan_as_category= False)
    # Calculate rate for each category with decay
    bb_processed = bb.groupby('SK_ID_BUREAU')[categorical_cols].mean().reset_index()
    # Min, Max, Count and mean duration of payments (months)
    agg = {'MONTHS_BALANCE': ['min', 'max', 'mean', 'size']}
    bb_processed = group_and_merge(bb, bb_processed, '', agg, 'SK_ID_BUREAU')
    del bb; gc.collect()
    return bb_processed

In [61]:
# group 코드
def group(df_to_agg, prefix, aggregations, aggregate_by= 'SK_ID_CURR'):
    agg_df = df_to_agg.groupby(aggregate_by).agg(aggregations)
    agg_df.columns = pd.Index(['{}{}_{}'.format(prefix, e[0], e[1].upper())
                               for e in agg_df.columns.tolist()])
    return agg_df.reset_index()

In [62]:
# group_and merge 코드
def group_and_merge(df_to_agg, df_to_merge, prefix, aggregations, aggregate_by= 'SK_ID_CURR'):
    agg_df = group(df_to_agg, prefix, aggregations, aggregate_by= aggregate_by)
    return df_to_merge.merge(agg_df, how='left', on= aggregate_by)

In [63]:
categorical_cols

In [64]:
bb_processed = bb.groupby('SK_ID_BUREAU')[categorical_cols].mean().reset_index()

In [65]:
bb.shape

In [66]:
bb_processed.shape

In [67]:
agg = {'MONTHS_BALANCE': ['min', 'max', 'mean', 'size']}

In [68]:
df_to_agg = bb
df_to_merge = bb_processed
prefix = ''
aggregations = agg
aggregate_by = 'SK_ID_BUREAU'

In [69]:
agg_df = df_to_agg.groupby(aggregate_by).agg(aggregations)
agg_df.head()

In [70]:
agg_df.columns.tolist()

In [71]:
# 인덱스를 바꿔주는 코드
agg_df.columns = pd.Index(['{}{}_{}'.format(prefix, e[0], e[1].upper())
                          for e in agg_df.columns.tolist()])

In [72]:
agg_df.reset_index().head()

In [73]:
agg_df.shape

위에 코드는 `group` 함수이며 밑에는 `group_and_merge` 함수입니다.

In [74]:
df_to_merge.merge(agg_df, how='left', on=aggregate_by)

In [75]:
bureau = bureau.merge(get_bureau_balance(bureau_balance_path, num_rows), how='left', on='SK_ID_BUREAU')

In [76]:
bureau['STATUS_12345'] = 0

for i in range(1, 6):
    bureau['STATUS_12345'] += bureau['STATUS_{}'.format(i)]

In [77]:
bureau['STATUS_12345'].unique()

Aggregate by number of months in balance and merge with bureau (loan length agg)  
balance에 있는 월들의 수를 기준으로 집계를 한 다음 bureau에 합쳐줍니다.

In [78]:
features = ['AMT_CREDIT_MAX_OVERDUE', 'AMT_CREDIT_SUM_OVERDUE', 'AMT_CREDIT_SUM',
        'AMT_CREDIT_SUM_DEBT', 'DEBT_PERCENTAGE', 'DEBT_CREDIT_DIFF', 'STATUS_0', 'STATUS_12345']

In [79]:
# 이자가 남은 기간을 기준으로 group한 다음 위의 numerical feautre의 평균을 뽑아냄
agg_length = bureau.groupby('MONTHS_BALANCE_SIZE')[features].mean()

In [80]:
bureau.groupby('MONTHS_BALANCE_SIZE')[features].mean()

In [81]:
# 컬럼 앞에 "LL_"을 붙여주는 코드
agg_length.rename({feat: 'LL_' + feat for feat in features}, axis=1, inplace=True)

In [82]:
bureau = bureau.merge(agg_length, how='left', on='MONTHS_BALANCE_SIZE')
del agg_length; gc.collect()

# general loans aggregations

general loans에 해당하는 bureau 데이터셋을 살펴보고, 이에 대한 통계치를 얻은 후 데이터셋을 합쳐줍니다.

bureau는 신용평가회사입니다. 이러한 데이터 셋의 특정한 컬럼에 대해서 mean, min, max 등의 통계 값을 집계해 새로운 컬럼을 만들어 줍니다.

In [83]:
BUREAU_AGG = {
    'SK_ID_BUREAU': ['nunique'], # id가 몇개의 기관에 있는지
    'DAYS_CREDIT': ['min', 'max', 'mean'],  # 대출기간
    'DAYS_CREDIT_ENDDATE': ['min', 'max'],
    'AMT_CREDIT_MAX_OVERDUE': ['max', 'mean'],
    'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
    'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
    'AMT_CREDIT_SUM_OVERDUE': ['max', 'mean', 'sum'],
    'DEBT_CREDIT_DIFF': ['mean', 'sum'],
    'MONTHS_BALANCE_MEAN': ['mean', 'var'],
    'MONTHS_BALANCE_SIZE': ['mean', 'sum'],
    # Categorical
    'STATUS_0':['mean'],
    'STATUS_1':['mean'],
    'STATUS_12345': ['mean'],
    'STATUS_C': ['mean'],
    'STATUS_X': ['mean'],
    'CREDIT_ACTIVE_Active': ['mean'],
    'CREDIT_ACTIVE_Closed': ['mean'],
    'CREDIT_ACTIVE_Sold': ['mean'],
    'CREDIT_TYPE_Consumer credit': ['mean'],
    'CREDIT_TYPE_Credit card': ['mean'],
    'CREDIT_TYPE_Car loan': ['mean'],
    'CREDIT_TYPE_Mortgage': ['mean'],
    'CREDIT_TYPE_Microloan': ['mean'],
    # Group by loan duration features (months)
    'LL_AMT_CREDIT_SUM_OVERDUE': ['mean'],
    'LL_DEBT_CREDIT_DIFF': ['mean'],
    'LL_STATUS_12345': ['mean'],
}

In [84]:
# group 코드
def group(df_to_agg, prefix, aggregations, aggregate_by= 'SK_ID_CURR'):
    agg_df = df_to_agg.groupby(aggregate_by).agg(aggregations)
    agg_df.columns = pd.Index(['{}{}_{}'.format(prefix, e[0], e[1].upper())
                               for e in agg_df.columns.tolist()])
    return agg_df.reset_index()

In [85]:
bureau.groupby('SK_ID_CURR').agg(BUREAU_AGG)

`SK_ID_CURR`은 고객의 ID입니다. nunique는 특정 고객의 대출 건수를 의미합니다. 각 통계값들은 이들 전체에 대한 수치를 나타냅니다.

agg_bureau라는 dataframe을 생성합니다. 앞으로 bureau에 관련해 생성한 agg feature는 여기다가 붙일 것입니다.

In [86]:
agg_bureau = group(bureau, 'BUREAU_', BUREAU_AGG)

In [87]:
agg_bureau

In [88]:
# bureau['CREDIT_ACTIVE_Closed'][:10]
# bureau['CREDIT_ACTIVE_Active'][:10]
# 서로 반대로 나타남


BUREAU_ACTIVE_AGG = {
    'DAYS_CREDIT': ['max', 'mean'],
    'DAYS_CREDIT_ENDDATE': ['min', 'max'],
    'AMT_CREDIT_MAX_OVERDUE': ['max', 'mean'],
    'AMT_CREDIT_SUM': ['max', 'sum'],
    'AMT_CREDIT_SUM_DEBT': ['mean', 'sum'],
    'AMT_CREDIT_SUM_OVERDUE': ['max', 'mean'],
    'DAYS_CREDIT_UPDATE': ['min', 'mean'],
    'DEBT_PERCENTAGE': ['mean'],
    'DEBT_CREDIT_DIFF': ['mean'],
    'CREDIT_TO_ANNUITY_RATIO': ['mean'],
    'MONTHS_BALANCE_MEAN': ['mean', 'var'],
    'MONTHS_BALANCE_SIZE': ['mean', 'sum'],
}

In [89]:
BUREAU_CLOSED_AGG = {
    'DAYS_CREDIT': ['max', 'var'],
    'DAYS_CREDIT_ENDDATE': ['max'],
    'AMT_CREDIT_MAX_OVERDUE': ['max', 'mean'],
    'AMT_CREDIT_SUM_OVERDUE': ['mean'],
    'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
    'AMT_CREDIT_SUM_DEBT': ['max', 'sum'],
    'DAYS_CREDIT_UPDATE': ['max'],
    'ENDDATE_DIF': ['mean'],
    'STATUS_12345': ['mean'],
}

Active and closed loans aggregations

active, closed의 통계값을 얻어서 agg_bureau에 붙여줌

In [90]:
active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
agg_bureau = group_and_merge(active, agg_bureau, 'BUREAU_ACTIVE_', BUREAU_ACTIVE_AGG)

In [91]:
closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
agg_bureau = group_and_merge(closed, agg_bureau, 'BUREAU_CLOSED_', BUREAU_CLOSED_AGG)

del active, closed; gc.collect()

In [92]:
BUREAU_LOAN_TYPE_AGG = {
    'DAYS_CREDIT': ['mean', 'max'],
    'AMT_CREDIT_MAX_OVERDUE': ['mean', 'max'],
    'AMT_CREDIT_SUM': ['mean', 'max'],
    'AMT_CREDIT_SUM_DEBT': ['mean', 'max'],
    'DEBT_PERCENTAGE': ['mean'],
    'DEBT_CREDIT_DIFF': ['mean'],
    'DAYS_CREDIT_ENDDATE': ['max'],
}

In [93]:
for credit_type in ['Consumer credit', 'Credit card', 'Mortgage', 'Car loan', 'Microloan']:
    type_df = bureau[bureau['CREDIT_TYPE_' + credit_type] == 1]
    prefix = 'BUREAU_' + credit_type.split(' ')[0].upper() + '_'
    agg_bureau = group_and_merge(type_df, agg_bureau, prefix, BUREAU_LOAN_TYPE_AGG)
    
    del type_df; gc.collect()

agg_bureau현재 agg_bureau의 컬럼 개수: 130개 (영상에서는 131개)

## Time frame aggregations

-180일괴 -360일, 즉 최근 1년과 6개월에 대한 컬럼을 만들어 합쳐줍니다.

In [94]:
BUREAU_TIME_AGG = {
    'AMT_CREDIT_MAX_OVERDUE': ['max', 'mean'],
    'AMT_CREDIT_SUM_OVERDUE': ['mean'],
    'AMT_CREDIT_SUM': ['max', 'sum'],
    'AMT_CREDIT_SUM_DEBT': ['mean', 'sum'],
    'DEBT_PERCENTAGE': ['mean'],
    'DEBT_CREDIT_DIFF': ['mean'],
    'STATUS_0': ['mean'],
    'STATUS_12345': ['mean'],
}

In [95]:
for time_frame in [6, 12]:
    prefix = 'BUREAU_LAST{}M_'.format(time_frame)
    time_frame_df = bureau[bureau['DAYS_CREDIT'] >= -30 * time_frame]
    agg_bureau = group_and_merge(time_frame_df, agg_bureau, prefix, BUREAU_TIME_AGG)
    
    del time_frame_df; gc.collect()

# Last loan max overdue

마지막 대출 연체액을 살펴보고 이에 대한 통계값을 얻어 컬럼을 만들어줍니다.

In [96]:
sort_bureau = bureau.sort_values(by=['DAYS_CREDIT'])
# last()는 리스트 등의 마지막 요소를 얻는다.
gr = sort_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_MAX_OVERDUE'].last().reset_index()
gr.rename({'AMT_CREDIT_MAX_OVERDUE': 'BUREAU_LAST_LOAN_MAX_OVERDUE'}, inplace=True)
agg_bureau = agg_bureau.merge(gr, on='SK_ID_CURR', how='left')

전체를 다 합친 것에 빛이 얼마인지, 전체를 합친 것에 비해 active debt(유동부채)의 비율이 얼마나 되는지에 대한 컬럼읆 만들어줍니다.

In [97]:
# Rations: total bebt/total credit and active loans debt/ active loans credit
agg_bureau['BUREAU_DEBT_OVER_CREDIT'] = \
    agg_bureau['BUREAU_AMT_CREDIT_SUM_DEBT_SUM']/agg_bureau['BUREAU_AMT_CREDIT_SUM_SUM']
agg_bureau['BUREAU_ACTIVE_DEBT_OVER_CREDIT'] = \
    agg_bureau['BUREAU_ACTIVE_AMT_CREDIT_SUM_DEBT_SUM']/agg_bureau['BUREAU_ACTIVE_AMT_CREDIT_SUM_SUM']

In [98]:
agg_bureau

In [99]:
df

In [100]:
df = df.merge(agg_bureau, on='SK_ID_CURR', how='left')

In [101]:
df.head()

# PREVIOUS PIPELINE

In [102]:
prev_path = '../input/home-credit-default-risk/previous_application.csv'
pay_path = '../input/home-credit-default-risk/installments_payments.csv'

prev = pd.read_csv(prev_path, nrows=num_rows)
pay = pd.read_csv(pay_path, nrows=num_rows)

In [103]:
prev.shape

In [104]:
# 원-핫 인코딩할 컬럼은 8개로 정했음.
ohe_columns = [
    'NAME_CONTRACT_STATUS', 'NAME_CONTRACT_TYPE', 'CHANNEL_TYPE',
    'NAME_TYPE_SUITE', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION',
    'NAME_PRODUCT_TYPE', 'NAME_CLIENT_TYPE']

In [105]:
prev, categorical_cols = one_hot_encoder(prev, ohe_columns, nan_as_category=False)

In [106]:
# Feature engineering : ratio and differnce

# 신청한 액수에 비해서 신용이 얼마나 있는지
prev['APPLICATION_CREDIT_DIFF'] = prev['AMT_APPLICATION'] - prev['AMT_CREDIT']
prev['APPLICATION_CREDIT_RATIO'] = prev['AMT_APPLICATION'] - prev['AMT_CREDIT']
prev['CREDIT_TO_ANNUITY_RATIO'] = prev['AMT_CREDIT'] / prev['AMT_ANNUITY']
prev['DOWN_PAYMENT_TO_CREDIT'] = prev['AMT_DOWN_PAYMENT'] / prev['AMT_CREDIT']

In [107]:
total_payment = prev['AMT_ANNUITY'] * prev['CNT_PAYMENT']

In [108]:
# 이자 계산
prev['SIMPLE_INTERESTS'] = (total_payment / prev['AMT_CREDIT'] -1) / prev['CNT_PAYMENT']

Active loans - approved and not complete yet (last_due 36243)  
신청은 했지만 아직 끝나지 않은 대출

In [109]:
# 승인된 대출
approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
# 아직 끝나지 않은 대출
active_df = approved[approved['DAYS_LAST_DUE'] == 365243]

`SK_ID_PREV`는 PREV 데이터프레임에 접근할 수 있는 키입니다.

In [110]:
active_pay = pay[pay['SK_ID_PREV'].isin(active_df['SK_ID_PREV'])]
active_pay.shape

In [111]:
active_pay_agg = active_pay.groupby('SK_ID_PREV')[['AMT_INSTALMENT', 'AMT_PAYMENT']].sum()
active_pay_agg

In [112]:
active_pay_agg.reset_index(inplace=True)

In [113]:
active_pay_agg['INSTALMENT_PAYMENT_DIFF'] = active_pay_agg['AMT_INSTALMENT'] - active_pay_agg['AMT_PAYMENT']

In [114]:
active_df = active_df.merge(active_pay_agg, on='SK_ID_PREV', how='left')
active_df.head()

피처 엔지니어링이 완료된 prev 데이터셋을 `SK_ID_CURR`로 그룹해 원래의 데이터 프레임에 붙여줍니다.

In [115]:
active_df

In [117]:
# Merge with active_df
active_df = active_df.merge(active_pay_agg, on='SK_ID_PREV', how = 'left')
active_df['REMAINING_DEBT'] = active_df['AMT_CREDIT'] - active_df['AMT_PAYMENT']
active_df['REPAYMENT_RATIO'] = active_df['AMT_PAYMENT'] / active_df['AMT_CREDIT']

In [118]:
df.merge(active_df.groupby('SK_ID_CURR').mean().reset_index(), on='SK_ID_CURR', how='left')

현재 컬럼 개수: 336  
동영상 컬럼 개수: 389

In [119]:
PREVIOUS_ACTIVE_AGG = {
    'SK_ID_PREV': ['nunique'],
    'SIMPLE_INTERESTS': ['mean'],
    'AMT_ANNUITY': ['max', 'sum'],
    'AMT_APPLICATION': ['max', 'mean'],
    'AMT_CREDIT': ['sum'],
    'AMT_DOWN_PAYMENT': ['max', 'mean'],
    'DAYS_DECISION': ['min', 'mean'],
    'CNT_PAYMENT': ['mean', 'sum'],
    'DAYS_LAST_DUE_1ST_VERSION': ['min', 'max', 'mean'],
    # Engineered features
    'AMT_PAYMENT': ['sum'],
    'INSTALMENT_PAYMENT_DIFF': ['mean', 'max'],
    'REMAINING_DEBT': ['max', 'mean', 'sum'],
    'REPAYMENT_RATIO': ['mean'],
}

In [120]:
active_agg_df = group(active_df, 'PREV_ACTIVE_', PREVIOUS_ACTIVE_AGG)
active_agg_df['PREV_ACTIVE_AMT_PAYMENT_SUM'] / active_agg_df['PREV_ACTIVE_AMT_CREDIT_SUM']
del active_pay, active_pay_agg, active_df; gc.collect()

In [121]:
# Change 365.243 values to nan (missing)
prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace=True)
prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace=True)
prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace=True)
prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace=True)
prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace=True)

In [122]:
# Days last due difference (scheduled x done) 기간이 얼마나 남았는지
prev['DAYS_LAST_DUE_DIFF'] = prev['DAYS_LAST_DUE_1ST_VERSION'] - prev['DAYS_LAST_DUE']
approved['DAYS_LAST_DUE_DIFF'] = approved['DAYS_LAST_DUE_1ST_VERSION'] - approved['DAYS_LAST_DUE']

In [123]:
categorical_cols[:5]

In [124]:
categorical_agg = {key: ['mean'] for key in categorical_cols}

In [125]:
categorical_agg

In [126]:
PREVIOUS_AGG = {
    'SK_ID_PREV': ['nunique'],
    'AMT_ANNUITY': ['min', 'max', 'mean'],
    'AMT_DOWN_PAYMENT': ['max', 'mean'],
    'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
    'RATE_DOWN_PAYMENT': ['max', 'mean'],
    'DAYS_DECISION': ['min', 'max', 'mean'],
    'CNT_PAYMENT': ['max', 'mean'],
    'DAYS_TERMINATION': ['max'],
    # Engineered features
    'CREDIT_TO_ANNUITY_RATIO': ['mean', 'max'],
    'APPLICATION_CREDIT_DIFF': ['min', 'max', 'mean'],
    'APPLICATION_CREDIT_RATIO': ['min', 'max', 'mean', 'var'],
    'DOWN_PAYMENT_TO_CREDIT': ['mean'],
}

In [127]:
# 두 개를 합치는 코드
{**PREVIOUS_AGG, **categorical_agg}

In [128]:
# group 코드
def group(df_to_agg, prefix, aggregations, aggregate_by= 'SK_ID_CURR'):
    agg_df = df_to_agg.groupby(aggregate_by).agg(aggregations)
    agg_df.columns = pd.Index(['{}{}_{}'.format(prefix, e[0], e[1].upper())
                               for e in agg_df.columns.tolist()])
    return agg_df.reset_index()

In [129]:
agg_prev = group(prev, 'PREV_', {**PREVIOUS_AGG, **categorical_agg})

In [130]:
# Merge active loans dataframe on agg_prev
agg_prev = agg_prev.merge(active_agg_df, how='left', on='SK_ID_CURR')
del active_agg_df; gc.collect()

In [131]:
PREVIOUS_APPROVED_AGG = {
    'SK_ID_PREV': ['nunique'],
    'AMT_ANNUITY': ['min', 'max', 'mean'],
    'AMT_CREDIT': ['min', 'max', 'mean'],
    'AMT_DOWN_PAYMENT': ['max'],
    'AMT_GOODS_PRICE': ['max'],
    'HOUR_APPR_PROCESS_START': ['min', 'max'],
    'DAYS_DECISION': ['min', 'mean'],
    'CNT_PAYMENT': ['max', 'mean'],
    'DAYS_TERMINATION': ['mean'],
    # Engineered features
    'CREDIT_TO_ANNUITY_RATIO': ['mean', 'max'],
    'APPLICATION_CREDIT_DIFF': ['max'],
    'APPLICATION_CREDIT_RATIO': ['min', 'max', 'mean'],
    # The following features are only for approved applications
    'DAYS_FIRST_DRAWING': ['max', 'mean'],
    'DAYS_FIRST_DUE': ['min', 'mean'],
    'DAYS_LAST_DUE_1ST_VERSION': ['min', 'max', 'mean'],
    'DAYS_LAST_DUE': ['max', 'mean'],
    'DAYS_LAST_DUE_DIFF': ['min', 'max', 'mean'],
    'SIMPLE_INTERESTS': ['min', 'max', 'mean'],
}

In [132]:
PREVIOUS_REFUSED_AGG = {
    'AMT_APPLICATION': ['max', 'mean'],
    'AMT_CREDIT': ['min', 'max'],
    'DAYS_DECISION': ['min', 'max', 'mean'],
    'CNT_PAYMENT': ['max', 'mean'],
    # Engineered features
    'APPLICATION_CREDIT_DIFF': ['min', 'max', 'mean', 'var'],
    'APPLICATION_CREDIT_RATIO': ['min', 'mean'],
    'NAME_CONTRACT_TYPE_Consumer loans': ['mean'],
    'NAME_CONTRACT_TYPE_Cash loans': ['mean'],
    'NAME_CONTRACT_TYPE_Revolving loans': ['mean'],
    }

In [133]:
# Aggregations for approved and refused loans
agg_prev = group_and_merge(approved, agg_prev, 'APPROVED_', PREVIOUS_APPROVED_AGG)
refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
agg_prev = group_and_merge(refused, agg_prev, 'REFUSED', PREVIOUS_REFUSED_AGG)
del approved, refused; gc.collect()

In [134]:
PREVIOUS_LOAN_TYPE_AGG = {
    'AMT_CREDIT': ['sum'],
    'AMT_ANNUITY': ['mean', 'max'],
    'SIMPLE_INTERESTS': ['min', 'mean', 'max', 'var'],
    'APPLICATION_CREDIT_DIFF': ['min', 'var'],
    'APPLICATION_CREDIT_RATIO': ['min', 'max', 'mean'],
    'DAYS_DECISION': ['max'],
    'DAYS_LAST_DUE_1ST_VERSION': ['max', 'mean'],
    'CNT_PAYMENT': ['mean'],
}

In [135]:
# Aggregation for Consumer loans and Cash loans
for loan_type in ['Consumer loans', 'Cash loans']:
    type_df = prev[prev['NAME_CONTRACT_TYPE_{}'.format(loan_type)] == 1]
    prefix = 'PREV_' + loan_type.split(" ")[0] + '_'
    agg_prev = group_and_merge(type_df, agg_prev, prefix, PREVIOUS_LOAN_TYPE_AGG)
    del type_df; gc.collect()

In [136]:
# Get the SK_ID_PREV for loans with late payments (days past due, 연체)
# 할부가 실제 갚아진 날 - 할부가 지불됐어야 했던 날
pay['LATE_PAYMENT'] = pay['DAYS_ENTRY_PAYMENT'] - pay['DAYS_INSTALMENT']

# 연체가 하루라도 있으면 1, 아니면 0
pay['LATE_PAYMENT'] = pay['LATE_PAYMENT'].apply(lambda x: 1 if x > 0 else 0)

# 연체가 있는 사람들의 id를 뽑음
dpd_id = pay[pay['LATE_PAYMENT'] > 0]['SK_ID_PREV'].unique()

In [137]:
PREVIOUS_LATE_PAYMENTS_AGG = {
    'DAYS_DECISION': ['min', 'max', 'mean'],
    'DAYS_LAST_DUE_1ST_VERSION': ['min', 'max', 'mean'],
    # Engineered features
    'APPLICATION_CREDIT_DIFF': ['min'],
    'NAME_CONTRACT_TYPE_Consumer loans': ['mean'],
    'NAME_CONTRACT_TYPE_Cash loans': ['mean'],
    'NAME_CONTRACT_TYPE_Revolving loans': ['mean'],
}

In [138]:
# Aggregations for loans with late payments
agg_dpd = group_and_merge(prev[prev['SK_ID_PREV'].isin(dpd_id)], agg_prev,
                          'PREV_LATE_', PREVIOUS_LATE_PAYMENTS_AGG)
del agg_dpd, dpd_id; gc.collect()

In [139]:
PREVIOUS_TIME_AGG = {
    'AMT_CREDIT': ['sum'],
    'AMT_ANNUITY': ['mean', 'max'],
    'SIMPLE_INTERESTS': ['mean', 'max'],
    'DAYS_DECISION': ['min', 'mean'],
    'DAYS_LAST_DUE_1ST_VERSION': ['min', 'max', 'mean'],
    # Engineered features
    'APPLICATION_CREDIT_DIFF': ['min'],
    'APPLICATION_CREDIT_RATIO': ['min', 'max', 'mean'],
    'NAME_CONTRACT_TYPE_Consumer loans': ['mean'],
    'NAME_CONTRACT_TYPE_Cash loans': ['mean'],
    'NAME_CONTRACT_TYPE_Revolving loans': ['mean'],
}

In [140]:
# Aggregations for loans in the last x months
# prev['DAYS_DECISION']: 대출이 이루어지기로 결정난 시점
for time_frame in [12, 24]:
    # 1년과 2년에 대해서만
    time_frame_df = prev[prev['DAYS_DECISION'] >= -30 * time_frame]
    prefix = 'PREV_LAST()M_'.format(time_frame)
    agg_prev = group_and_merge(time_frame_df, agg_prev, prefix, PREVIOUS_TIME_AGG)
    del time_frame_df; gc.collect()

In [141]:
agg_prev.head()

# POS-CASH-PIPELINE

""" Process POS_CASH_balance.csv and return a pandas dataframe. """

In [142]:
pos_path = '../input/home-credit-default-risk/POS_CASH_balance.csv'
pos = pd.read_csv(pos_path, nrows=num_rows)
pos, categorical_cols = one_hot_encoder(pos, nan_as_category=False)

In [143]:
# Flag months with late payment
# 연체된 사람들에 대한 정보를 모음
pos['LATE_PAYMENT'] = pos['SK_DPD'].apply(lambda x:1 if x > 0 else 0)

In [144]:
pos['SK_DPD'].unique()

In [145]:
POS_CASH_AGG = {
    'SK_ID_PREV': ['nunique'],
    'MONTHS_BALANCE': ['min', 'max', 'size'],
    'SK_DPD': ['max', 'mean', 'sum', 'var'],
    'SK_DPD_DEF': ['max', 'mean', 'sum'],
    'LATE_PAYMENT': ['mean']
}

In [146]:
# Aggregate by SK_ID_CURR
categorical_agg = {key: ['mean'] for key in categorical_cols}
pos_agg = group(pos, 'POS_', {**POS_CASH_AGG, **categorical_agg})

In [147]:
{**POS_CASH_AGG, **categorical_agg}

In [148]:
# Sort and group by SK_ID_PREV
sort_pos = pos.sort_values(by = ['SK_ID_PREV', 'MONTHS_BALANCE'])
gp = sort_pos.groupby('SK_ID_PREV')

df = pd.DataFrame()

# groupby 된 것의 첫 번째 값을 가져와 할당. row 개수> 30000개 -> 28559개
df['SK_ID_CURR'] = gp['SK_ID_CURR'].first()
df['MONTHS_BALANCE_MAX'] = gp['MONTHS_BALANCE'].max()

In [149]:
sort_pos.shape

In [150]:
sort_pos['SK_ID_CURR'].nunique()

In [151]:
sort_pos['SK_ID_PREV'].nunique()

In [152]:
gp.mean().shape

In [153]:
df.shape

In [154]:
# Percentage of previous loans completed and completed before initial term
# NAME_CONTRACT_STATUS: 당월 계약 현황, NAME_CONTRACT_STATUS_Completed: 할부가 끝났는지

df['POS_LOAN_COMPLETED_MEAN'] = gp['NAME_CONTRACT_STATUS_Completed'].mean()

# 기간의 첫 날과 마지막 날을 빼줍니다. = 전체 할부 일 수
df['POS_COMPLETED_BEFORE_MEAN'] = gp['CNT_INSTALMENT'].first() - gp['CNT_INSTALMENT'].last()

# 두 개 이상의 column에 lambda 조건을 적용하고 싶으면 데이터프레임에 바로 apply를 붙이고 axis를 주면 됩니다.
df['POS_COMPLETED_BEFORE_MEAN'] = df.apply(lambda x: 1 if x['POS_COMPLETED_BEFORE_MEAN'] > 0
                                           and x['POS_LOAN_COMPLETED_MEAN'] > 0 else 0, axis = 1)

In [155]:
sort_pos['NAME_CONTRACT_STATUS_Completed'].mean()

In [156]:
df['POS_LOAN_COMPLETED_MEAN'].head(15)

In [157]:
# Number of remaining installments (future installments) and percentage from total
# 전체 중 할부가 남은 수와 퍼센티지

# 가장 최신의 할부 기간
df['POS_REMAINING_INSTALMENTS'] = gp['CNT_INSTALMENT_FUTURE'].last()
df['POS_REMAINING_INSTALMENTS_RATIO'] = gp['CNT_INSTALMENT_FUTURE'].last() / gp['CNT_INSTALMENT'].last()

In [158]:
# Group by SK_ID_CURR and merge
df_gp = df.groupby('SK_ID_CURR').sum().reset_index()
df_gp.drop(['MONTHS_BALANCE_MAX'], axis = 1, inplace = True)
pos_agg = pd.merge(pos_agg, df_gp, on = 'SK_ID_CURR', how = 'left')
del df, gp, df_gp, sort_pos; gc.collect()

In [159]:
def do_sum(df, group_cols, counted, agg_name):
    gp = df[group_cols + [counted]].groupby(group_cols)[counted].sum().reset_index().rename(
        columns = {counted: agg_name})
    df = df.merge(gp, on = group_cols, how = 'left')
    del gp
    gc.collect()
    return df

In [160]:
pos.head()

In [161]:
# Percentage of late payments for the 3 most recent applications
pos = do_sum(pos, ['SK_ID_PREV'], 'LATE_PAYMENT', 'LATE_PAYMENT_SUM')

In [162]:
pos[['SK_ID_PREV'] + ['LATE_PAYMENT']].head()

In [163]:
pos.head()

groupby를 한다고 해도 각 행의 index 값은 변하지 않습니다. 따라서 idxmax() 메서드를 통해 가장 큰(밑에 있는), 최신의 인덱스를 반환합니다.

In [164]:
# 최근 3개의 balance 정보를 가져보려면 groupby를 여러번 해야합니다.

# last months of each applications
# idxmax(): 최대값을 가지는 인덱스 레이블을 출력한다.
last_month_df = pos.groupby('SK_ID_PREV')['MONTHS_BALANCE'].idxmax()

In [165]:
# groupby를 하면 자동으로 오름차순 정렬이 됩니다.
pos.groupby('SK_ID_PREV')['MONTHS_BALANCE'].idxmax()

In [166]:
for i, ele in pos.groupby('SK_ID_PREV')['MONTHS_BALANCE']:
    break

In [167]:
ele

`SK_ID_PREV`가 1000013인 사람의 last month의 인덱스는 29474입니다.

In [168]:
for i, ele in pos.groupby('SK_ID_PREV'):
    break

In [169]:
# 가장 최신의 것을 가져오고
ele['MONTHS_BALANCE']

In [170]:
# 가장 최신의 것의 인덱스를 뽑아냄
ele['MONTHS_BALANCE'].idxmax()

In [171]:
# last month의 index를 가져옵니다. (iloc를 사용해 해당 인덱스의 정보를 가져옴)
pos.iloc[29474]

In [172]:
# MONTHS_BALBANCE: 신청일 대비 잔액월수

sort_pos = pos.sort_values(by=['SK_ID_PREV', 'MONTHS_BALANCE'])

# Most recent applications (last 3) 하나만 있으면 하나, 세개 있으면 세개를 가져옵니다.
gp = sort_pos.iloc[last_month_df].groupby('SK_ID_CURR').tail(3)
gp.head()

In [173]:
# 최신거 3개 뽑은 것의 평균값
gp_mean = gp.groupby('SK_ID_CURR').mean().reset_index()
pos_agg = pd.merge(pos_agg, gp_mean[['SK_ID_CURR', 'LATE_PAYMENT_SUM']], on = 'SK_ID_CURR', how = 'left')

In [174]:
gp_mean.head()